# Adicionando as ferramentas ao modelo

A última etapa é realmente chamarmos essas tools quando o modelo solicitar, criando assim uma aplicação funcional que combina modelo com ferramentas criadas pelo usuário.

## Recriando as tools

Vamos primeiro recriar as tools da última aula.

## Criando roteamento para rodar ferramenta

Chamamos de roteamento o processo de análise da output de um modelos que possui acesso a ferramntas. Noste processo, precisamos entender qual é o tipo de saída que o modelo está nos fornecendo. Caso seja uma saída já com a resposta final do modelo, podemos retorná-la ao usuário, caso o modelo esteja solicitando a chamado de uma ferramenta, temos que chamar essa ferramenta e mostrar o resultado ao usuário.

### Adicionando OpenAIFunctionsAgentOutputParser

Para auxiliar neste processo, podemos utilizar o OpenAIFunctionsAgentOutputParser. Ele processa a saída de um modelo da OpenAi que possui ferramentas e determina o estado da mensagem devolvida.

O retorno será um AgentAction caso necessite que uma ferramenta seja executada.

AgentActionMessageLog(tool='busca_wikipedia', tool_input={'query': 'Isaac Asimov'}, log="\nInvoking: `busca_wikipedia` with `{'query': 'Isaac Asimov'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 132, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a6054468-b0da-45ba-af68-aa2ab5c1ed44-0', usage_metadata={'input_tokens': 132, 'output_tokens': 21, 'total_tokens': 153, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])

'busca_wikipedia'

{'query': 'Isaac Asimov'}

E um AgentFinish, caso a mensagem esteja finalizada.

AgentFinish(return_values={'output': 'Olá! Como posso te ajudar hoje?'}, log='Olá! Como posso te ajudar hoje?')

'Olá! Como posso te ajudar hoje?'

### Rodando as ferramentas

Podemos criar uma função simples de roteamento para lidar com os dois estados possíveis:

'Olá! Como posso te ajudar hoje?'

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro